<a href="https://colab.research.google.com/github/PTC-Education/PTC-API-Playground/blob/main/AL5D_Digital_Twin_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AL5D PLTW Onshape Digital Twin
This notebook allows you to control the AL5D robot arm using a digtial twin in Onshape. Derivation of the inverse kinematics [can be found here](https://colab.research.google.com/github/PTC-Education/PTC-API-Playground/blob/main/Inverse_Kinematics.ipynb).

### Connect to Local Runtime

In [ ]:
jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0

### Set up serial connection to Robot Arm

In [ ]:
import serial
import sys
import glob
import time

def serial_ports():
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
        for port in ports:
            if 'usb' in port:
                guess = port
        return guess
    else:
        raise EnvironmentError('Unsupported platform')
    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return ports

def serial_write(string):
    ser.write(string + b'\r\n')
    time.sleep(0.1)
    while ser.in_waiting:  
        print(ser.read(ser.in_waiting).decode())
        

Should return "True" if connection is successfully opened (may need to unplug/replug USB cable if it is not connecting)

In [ ]:
port = serial_ports()
ser = serial.Serial(
    port=port,
    baudrate=9600
)

ser.isOpen()

True

Test the connection by writing to the hand motor

In [ ]:
ser.write(b'#4 P1500\r')

time.sleep(1)
while ser.in_waiting:  
    ser.read(ser.in_waiting)

# Connect to Onshape

In [ ]:
!pip install onshape-client
from onshape_client.client import Client
import json
base = 'https://cad.onshape.com' # change this if you're using a document in an enterprise (i.e. "https://ptc.onshape.com")

In [ ]:
%run '~/Documents/colabkeys.py'
client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})
print('client configured')

client configured


### Define functions for getting the mate values from Onshape and setting the mate values in Onshape

In [ ]:
def getMateValues():
  RobotData = []
  RobotData = [0 for i in range(6)]
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'

  # https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/2f52bf4870f9d7ddc900b4de
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'
  eid = '2f52bf4870f9d7ddc900b4de'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  fullResponse = json.loads(response.data)

  for i in range(len(fullResponse["mateValues"])):
    if fullResponse['mateValues'][i]['mateName'] == "Base":
      RobotData[0] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))
    elif fullResponse['mateValues'][i]['mateName'] == "Shoulder":
      RobotData[1] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))
    elif fullResponse['mateValues'][i]['mateName'] == "Elbow":
      RobotData[2] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))-70
    elif fullResponse['mateValues'][i]['mateName'] == "Wrist":
      RobotData[3] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,2500,500))+50
    elif fullResponse['mateValues'][i]['mateName'] == "Hand":
      RobotData[4] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))
    elif fullResponse['mateValues'][i]['mateName'] == "Gripper":
      RobotData[5] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))
  
  return RobotData

def setMateValues(baseAngle,shoulderAngle,elbowAngle,wristAngle,handAngle,gripperAngle):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'

  # https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/2f52bf4870f9d7ddc900b4de
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'
  eid = '2f52bf4870f9d7ddc900b4de'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  fullResponse = json.loads(response.data)
  
  for i in range(len(fullResponse["mateValues"])):
    if fullResponse['mateValues'][i]['mateName'] == "Base":
      print(fullResponse['mateValues'][i])
      fullResponse['mateValues'][i]['rotationZ'] = baseAngle
      print(fullResponse['mateValues'][i])

  method = 'POST'

  params = {}
  payload = fullResponse
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}
  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  for i in range(len(fullResponse["mateValues"])):
    if fullResponse['mateValues'][i]['mateName'] == "Shoulder":
      fullResponse['mateValues'][i]['rotationZ'] = shoulderAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Elbow":
      fullResponse['mateValues'][i]['rotationZ'] = elbowAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Wrist":
      fullResponse['mateValues'][i]['rotationZ'] = wristAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Hand":
      fullResponse['mateValues'][i]['rotationZ'] = handAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Gripper":
      fullResponse['mateValues'][i]['rotationZ'] = gripperAngle

  method = 'POST'

  params = {}
  payload = fullResponse
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}
  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  # The command below prints the entire JSON response from Onshape
  print(response.status)

def translate(value, leftMin, leftMax, rightMin, rightMax):
  # Figure out how 'wide' each range is
  leftSpan = leftMax - leftMin
  rightSpan = rightMax - rightMin

  # Convert the left range into a 0-1 range (float)
  valueScaled = float(value - leftMin) / float(leftSpan)

  # Convert the 0-1 range into a value in the right range.
  return rightMin + (valueScaled * rightSpan)

Send digital twin to its home position (all joints at 90 degrees)

In [ ]:
setMateValues(math.pi/2,math.pi/2,math.pi/2,math.pi/2,math.pi/2,math.pi/2)

{'jsonType': 'Revolute', 'rotationZ': 1.57079632679486, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': 1.5707963267948966, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200


In [ ]:
# numpy.arange([start, ]stop, [step, ], dtype=None) -> numpy.ndarray
tester = [0 for i in range(6)]
empty = [0 for i in range(6)]
tester = np.vstack([tester,empty])
try:
  tester[2][3] = 1
except:
  tester = np.vstack([tester,empty])
print(tester)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


# Inverse Kinematics

In [ ]:
import math
import numpy
DegToRad = math.pi/180
RadToDeg = 180/math.pi

FloorToFirstJoint = 2.61*0.0254
UpperArmLength = 5.74*0.0254
ForeArmLength = 7.23*0.0254
GripperLength = 4.43*0.0254

def GetJointAngles(TCPx,TCPy,TCPz,WristAngleIn):
  BaseAngle = math.atan2(TCPx,TCPy)

  TCPq = math.sqrt(TCPx**2 + TCPy**2)
  TCPp = TCPz - FloorToFirstJoint

  WristQ = TCPq - GripperLength*math.cos(WristAngleIn*DegToRad)
  WristP = TCPp + GripperLength*math.sin(WristAngleIn*DegToRad)

  D2 = math.sqrt(WristP**2 + WristQ**2)
  print(WristQ,WristP)

  ElbowAngle = math.acos((WristP**2 + WristQ**2 - UpperArmLength**2 - ForeArmLength**2)/(2*UpperArmLength*ForeArmLength))

  a1 = math.atan2(ForeArmLength*math.sin(ElbowAngle),UpperArmLength + ForeArmLength*math.cos(ElbowAngle))
  a2 = math.atan2(WristP,WristQ)
  ShoulderAngle = a1 + a2

  WristAngleOut = ShoulderAngle - ElbowAngle + WristAngleIn*DegToRad + math.pi/2

  print(numpy.dot([BaseAngle, ShoulderAngle, ElbowAngle, WristAngleOut],RadToDeg))
  return [BaseAngle, ShoulderAngle, ElbowAngle, WristAngleOut]

In [ ]:
setMateValues(GetJointAngles(0.2,0.2,0.2,0),math.pi/2,0)

0.17032071247461908 0.13370600000000002
[45.         95.09984984 98.69417921 86.40567063]
200


## Get Checkoint position

In [ ]:
def getCheckpointPos():
  import array
  base = 'https://cad.onshape.com'
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid'

  # https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/2f52bf4870f9d7ddc900b4de
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'
  eid = '2f52bf4870f9d7ddc900b4de'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))

  ## Specify number of checkpoints
  NumberOfCheckpoints = 1

  checkpointId = []
  checkpointId = ["id" for i in range(NumberOfCheckpoints)]

  checkpointPos = []
  checkpointPos = [[0,0,0] for i in range(NumberOfCheckpoints)]

  checkpointNum = 0
  for i in range(len(parsed['rootAssembly']['instances'])):
    if "Checkpoint" in parsed['rootAssembly']['instances'][i]['name']:
      checkpointId[checkpointNum] = parsed['rootAssembly']['instances'][i]['id']
      checkpointNum += 1
      if checkpointNum == NumberOfCheckpoints:
        break
  # print(checkpointId)

  checkpointNum = 0
  for i in range(len(parsed['rootAssembly']['occurrences'])):
    for j in range(NumberOfCheckpoints):
      if parsed['rootAssembly']['occurrences'][i]['path'][0] == checkpointId[j] and len(parsed['rootAssembly']['occurrences'][i]['path'])==1:
        checkpointPos[j] = [parsed['rootAssembly']['occurrences'][i]['transform'][3],parsed['rootAssembly']['occurrences'][i]['transform'][7],parsed['rootAssembly']['occurrences'][i]['transform'][11]]
  return checkpointPos

In [ ]:
def getTCPPos():
  import array
  base = 'https://cad.onshape.com'
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid'

  # https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/2f52bf4870f9d7ddc900b4de
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'
  eid = '2f52bf4870f9d7ddc900b4de'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  # print(json.dumps(parsed, indent=4, sort_keys=True))

  for i in range(len(parsed['rootAssembly']['instances'])):
    if "TCP" in parsed['rootAssembly']['instances'][i]['name']:
      TCPid = parsed['rootAssembly']['instances'][i]['id']
      break

  for i in range(len(parsed['rootAssembly']['occurrences'])):
    if parsed['rootAssembly']['occurrences'][i]['path'][0] == TCPid and len(parsed['rootAssembly']['occurrences'][i]['path'])==1:
      TCPpos = [parsed['rootAssembly']['occurrences'][i]['transform'][3],parsed['rootAssembly']['occurrences'][i]['transform'][7],parsed['rootAssembly']['occurrences'][i]['transform'][11]]
  return TCPpos
print(getTCPPos())
print(getCheckpointPos())

[0.14385380726199481, 0.060439793367692715, 0.3532739781232383]
[[0.14464893895175013, 0.049074929578456876, 0.3572741929778415]]


In [ ]:
pos = getCheckpointPos()[0]
ikArray = GetJointAngles(pos[0],pos[1],pos[2],0)
setMateValues(ikArray[0],ikArray[1],ikArray[2],ikArray[3],math.pi/2,0)

0.09805709040665586 0.12007882476994938
[ 93.0789431  125.94020741 125.30433915  90.63586826]
{'jsonType': 'Revolute', 'rotationZ': 1.9790730376617336, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': 1.6245340213771557, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200


In [ ]:
LastJointArray = [1500,1500,1500,1500,1500,1500]

In [ ]:
import time
import numpy as np

smoothingThreshold = 100
steps = 15
delay = 0.05

while True:
  pos = getCheckpointPos()[0]
  ikArray = GetJointAngles(pos[0],pos[1],pos[2],0)
  setMateValues(ikArray[0],ikArray[1],ikArray[2],ikArray[3],math.pi/2,0)
  JointArray = getMateValues()

  # numpy.arange([start, ]stop, [step, ], dtype=None) -> numpy.ndarray
  ndPosArray = [0 for i in range(6)]
  empty = [0 for i in range(6)]
  ndPosArray = np.vstack([ndPosArray,empty])
  # try:
  #   tester[2][3] = 1
  # except:
  #   tester = np.vstack([tester,empty])

  # print(JointArray)
  for i in range(len(JointArray)):
    if LastJointArray[i] - JointArray[i] < 0:
      posArray = np.array(range(LastJointArray[i],JointArray[i],steps))
    else:
      posArray = np.array(range(LastJointArray[i],JointArray[i],-steps))
    for j in range(len(posArray)):
      try:
        ndPosArray[j][i] = posArray[j]
      except:
        ndPosArray = np.vstack([ndPosArray,empty])
        ndPosArray[j][i] = posArray[j]

  for x in ndPosArray:
    print(x)
    for i in range(len(x)):
      if x[i] != 0:
        command = '#'+str(i)+' P'+str(x[i])+'\r'
        ser.write(command.encode())
        time.sleep(delay)
  LastJointArray = JointArray
  time.sleep(0.25)